# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
openai = OpenAI()


API key looks good so far


In [23]:

# set up environment

OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [24]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [25]:
# Get gpt-4o-mini to answer, with streaming

def stream_answer(model, question):
    stream = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant who explains Python code clearly in Polish."},
            {"role": "user", "content": question},
    
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [26]:
stream_answer(MODEL_GPT, question)

Ten fragment kodu wykorzystuje konstrukcję `yield from`, która jest częścią generatora w Pythonie. Przyjrzyjmy się szczegółowo, co się dzieje w tym kodzie.

1. **Zrozumienie `yield from`**:
   - `yield from` to instrukcja, która pozwala na wygenerowanie wartości z innego iterowalnego obiektu. Oznacza to, że generator przejmuje wartości z obiektu, z którego korzysta.
  
2. **Zrozumienie wyrażenia komprehendującego**:
   - `{book.get("author") for book in books if book.get("author")}` to wyrażenie komprehendujące zbioru, które tworzy zbiór autorów książek.
   - `book.get("author")` zwraca wartość przypisaną do klucza `"author"` w obiekcie `book`. Jeśli dany klucz nie istnieje, metoda `.get()` zwróci `None`.
   - `for book in books` iteruje po każdym obiekcie `book` w kolekcji `books`.
   - `if book.get("author")` filtruje książki, które mają przypisanego autora, a więc tylko książki z istniejącym kluczem `"author"` zostaną uwzględnione w finalnym zbiorze.

3. **Podsumowując**:
   - Kod ten tworzy zbiór unikalnych autorów książek z listy `books`, gdzie tylko książki mające przypisanego autora są brane pod uwagę.
   - `yield from` następnie "wydaje" każdy z tych autorów jeden po drugim, gdy generator jest używany.

### Dlaczego to jest użyteczne?
- Używanie `yield from` pozwala na łatwy i efektywny sposób na przekazywanie wartości z jednego generatora (w tym przypadku zbioru autorów) do wywołania generatora, co może być przydatne w sytuacjach, gdy chcemy uzyskać dostęp do przefiltrowanych danych bez tworzenia oddzielnej struktury danych.
- Dzięki zastosowaniu zbioru, otrzymujemy także unikalne wartości autorów, co może być istotne, gdy nie chcemy mieć duplikatów.

In [27]:
def stream_answer(model, question):
    stream = ollama.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant who explains Python code clearly in Polish."},
            {"role": "user", "content": question},
    
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [29]:
stream_answer(MODEL_LLAMA, question)

Ten kod wyświetla listę autorów książek z collectionu `books`, który jest przypisany do zmiennej `book`. Zawiera on dwie warunki:

1. Wyświetlowanie tylko tych książek, które mają własny "author" - jest to składnik klucza `"author"` wywołany przez metodę `get()` i sprawdzonego jako prawidłowy.
2. Zmiana obiektu `book` na generator.

Dwa elementy tej logiki dają efekt, który jest opisany w kodzie:

`yield from {book.get("author") for book in books if book.get("author")}`

Jego skuteczność polega na tym, że:

- Lista contains() lub any() działa na generatorze i wyświetla tylko elementy spełniające warunki zagnieżdżone w `for` loop.
 
-  `yield from` wywołuje metody generate() dla generatora, zwalniając go od obowiązku powtarzanego wywoływania `next()` na każdej kolejnej iteracji.